# Game of WORDLE

Wordle is a popular word guessing game. Here are the rules of the game:

The game begins with a randomly selected five-letter word.
The player must guess the word within ten attempts.

After each guess, the player will receive feedback in the form of a list of integers of length 5, containing either 0,1, or 2 (eg [0,0,1,2,1] or [1,1,2,0,1])

If the ith integer is:
*  0, then the ith letter of the guess is not present in the word
* 1, then the ith letter of the guess is correctly positioned
* 2, then the ith letter of the guess is present in the word but is misplaced.

The player can use this feedback to refine their guesses and work towards solving the word.
The game ends when the player correctly guesses the word or uses up all ten attempts.


**You have been provided with most of the code to run the wordle game, to take in your guesses and to provide feedback. Your task is to code the ```generate_guess()``` function which takes as input a feedback which is a list of integers of length 5 containing either 0, 1 or 2 as explained earlier and returns the best possible guess of a 5 letter word.**

The list of words to choose from is stored in the variable ```self.words```, and the correct word is guaranteed to belong to this list.

The game runs your algorithm for 10 different words and prints out your guesses and whether you got the correct answer. Note that, an average human takes just above 4 guesses to solve a wordle. With 10 allowed guesses your algorithm has infact been given a lot of headroom. Try to do better than 10 and optimize as much as you can.

In [1]:
from google.colab import drive
import sys
drive.mount('/content/drive/', force_remount = True)
sys.path.append('/content/drive/MyDrive/wordle')
from utils import *

Mounted at /content/drive/


In [2]:
import math
import requests

class Solver(Wordle):
    def __init__(self):
        super().__init__()

    possible_ans_list = []
    # The compiler wasn't able to access self.words (Error: name 'self' is not defined.)
    # So, I manually inputted the list into another variable.
    response = requests.get('https://raw.githubusercontent.com/charlesreid1/five-letter-words/master/sgb-words.txt')
    data = response.text
    possible_ans_list = data.split('\n')
    possible_ans_list.remove('')

    def reset_list():
        Solver.guess = "sales"
        Solver.possible_ans_list = Solver.data.split('\n')
        Solver.possible_ans_list.remove('')

    def get_possible_words(self,word, output, possible_ans_list,flag):
        alpha0 = []
        alpha1 = {}
        alpha2 = {}
        for i in range(0,5):
            if output[i] == 0:
                alpha0.append(word[i])
        for i in range(0,5):
            if output[i] == 1:
                alpha1[word[i]] = i
        for i in range (0,5):
            if output[i] == 2:
                alpha2[word[i]] = i

        remove_words = []
        for search0 in possible_ans_list:
            for ch in search0:
                if ch in alpha0:
                    remove_words.append(search0)
                    break
        for search1 in possible_ans_list:
            for j in alpha1:
                if search1[alpha1[j]] != j:
                    remove_words.append(search1)
                    break
        for search2 in possible_ans_list:
            for ch in alpha2:
                if (ch not in search2):
                    remove_words.append(search2)
                    break

        if flag == 1:       # When updation in list required.
            for search2 in possible_ans_list:
                for j in alpha2:
                    if search2[alpha2[j]] == j:
                        remove_words.append(search2)
                        break
            for i in remove_words:
                if i in possible_ans_list:
                    possible_ans_list.remove(i)
            return possible_ans_list
        if flag == 0:       # When entropy calculation is done.
            return (len(possible_ans_list)-len(set(remove_words)))


    fdbk_list = []
    for a in [0,1,2]:
        for b in [0,1,2]:
            for c in [0,1,2]:
                for d in [0,1,2]:
                    for e in [0,1,2]:
                        fdbk = [a,b,c,d,e]
                        fdbk_list.append(fdbk)

    guess = "sales"
    def generate_guess(self, feedback=None):
        if feedback == None:
            Solver.reset_list()
            return Solver.guess
        else:
            Solver.get_possible_words(self,Solver.guess,feedback,Solver.possible_ans_list,1)
            max_entropy = 0
            max_entropy_word = ""
            for chrt in Solver.possible_ans_list:
                total_entropy = 0
                for fdbk in Solver.fdbk_list:
                    try:
                        p_x = Solver.get_possible_words(self,chrt,fdbk,Solver.possible_ans_list,0)/len(Solver.possible_ans_list)
                        entropy = (p_x)*(math.log((1/p_x),2))
                        total_entropy += entropy
                    except ZeroDivisionError:
                        continue
                if total_entropy >= max_entropy:
                    max_entropy_word = chrt
                    max_entropy = total_entropy
            Solver.guess = max_entropy_word
            return Solver.guess


In [ ]:
game = Game(Solver, N=50)
game.run()

Simulating for  50  words.

WORD #[ 1 / 50 ]
	Guess # 1 	: sales 	Feedback:  [0, 0, 1, 1, 0]
	Guess # 2 	: oiled 	Feedback:  [1, 0, 1, 1, 1]
	Guess # 3 	: ogled 	Feedback:  [1, 1, 1, 1, 1]
	
Success.	Correct word:  ogled 	Number of guesses:  3


WORD #[ 2 / 50 ]
	Guess # 1 	: sales 	Feedback:  [2, 2, 2, 0, 1]
	Guess # 2 	: flaps 	Feedback:  [0, 1, 1, 2, 1]
	Guess # 3 	: plats 	Feedback:  [1, 1, 1, 0, 1]
	Guess # 4 	: plays 	Feedback:  [1, 1, 1, 1, 1]
	
Success.	Correct word:  plays 	Number of guesses:  4


WORD #[ 3 / 50 ]
	Guess # 1 	: sales 	Feedback:  [0, 0, 0, 1, 0]
	Guess # 2 	: eider 	Feedback:  [2, 0, 2, 1, 2]
	Guess # 3 	: cored 	Feedback:  [1, 0, 1, 1, 1]
	Guess # 4 	: cured 	Feedback:  [1, 1, 1, 1, 1]
	
Success.	Correct word:  cured 	Number of guesses:  4


WORD #[ 4 / 50 ]
	Guess # 1 	: sales 	Feedback:  [0, 0, 0, 2, 0]
	Guess # 2 	: eerie 	Feedback:  [2, 2, 2, 0, 1]
	Guess # 3 	: prone 	Feedback:  [0, 1, 0, 0, 1]
	Guess # 4 	: brute 	Feedback:  [0, 1, 1, 2, 1]
	Guess # 5 	:

# References

1. 3Blue1Brown's Youtube Video: https://youtu.be/v68zYyaEmEA
2. Information Theory applied to Wordle:
https://towardsdatascience.com/information-theory-applied-to-wordle-b63b34a6538e